<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Trade_AI/blob/main/NHITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clean, pinned stack known to work with NHITS + NeuralForecast
# This cell will RESTART the runtime at the end.

# 1) Remove preinstalled collisions we don't need
!pip -q uninstall -y torchvision torchaudio -y

# 2) Tooling
!pip -q install -U pip setuptools wheel

# 3) Pinned versions for stability
!pip -q install torch==2.3.1 pytorch-lightning==2.3.0 neuralforecast==1.6.4 pandas==2.2.2 pyarrow==16.1.0 fastparquet==2024.5.0

# 4) Show versions
import platform, importlib, torch
import pytorch_lightning as lightning
print("Python:", platform.python_version())
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Lightning:", lightning.__version__)
print("neuralforecast:", importlib.import_module("neuralforecast").__version__)

# 5) Hard restart to finalize clean env
import os
print("Restarting runtime...")
os.kill(os.getpid(), 9)

/usr/local/lib/python3.12/dist-packages/lightning_fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


Python: 3.12.12
Torch: 2.3.1+cu121 | CUDA: True
Lightning: 2.3.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Project paths (adjust if needed)
BASE_DIR = "/content/drive/MyDrive/ai4trade"
DATA_DIR = f"{BASE_DIR}/data"
FEATS_DIR = f"{DATA_DIR}/features"
MODELS_DIR = f"{BASE_DIR}/models/nhits"
PRED_DIR = f"{BASE_DIR}/predictions"
OOF_DIR = f"{PRED_DIR}/oof"
FC_DIR = f"{PRED_DIR}/forecast"
LOG_DIR = f"{BASE_DIR}/logs"

import os
for d in [DATA_DIR, FEATS_DIR, MODELS_DIR, PRED_DIR, OOF_DIR, FC_DIR, LOG_DIR]:
    os.makedirs(d, exist_ok=True)

print("Paths set.")

Mounted at /content/drive
Paths set.


In [2]:
import os, gc, json, math, warnings, platform
import pandas as pd
import numpy as np
from datetime import datetime

import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Python:", platform.python_version(), "| Torch:", torch.__version__, "| Device:", DEVICE)

# Quiet some nuisance warnings
warnings.filterwarnings("ignore", message="pkg_resources is deprecated")

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS  # <-- correct class name

# sMAPE with epsilon floor
EPS = 1.0
def smape(y_true, y_pred, eps=EPS):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    numer = 2.0 * np.abs(y_true - y_pred)
    denom = np.maximum(np.abs(y_true) + np.abs(y_pred), eps)
    return float(np.mean(numer / denom))

RUN_ID = datetime.now().strftime("%Y%m%d_%H%M")
MODEL_NAME = "nhits_h2"

# Horizon policy
H = 2
folds = [
    ('2023-07-01', '2023-09-01'),
    ('2024-01-01', '2024-03-01'),
    ('2024-07-01', '2024-09-01'),
]
FINAL_TRAIN_END = '2025-08-01'
FINAL_FORECAST_MONTH = '2025-10-01'

Python: 3.12.12 | Torch: 2.3.1+cu121 | Device: cuda


In [3]:
# Expect: features_train_h2.parquet / features_test_h2.parquet with cols:
# ['origin','destination','hs6','hs4','trade_flow','month','y', <engineered features>]

train_path = f"{FEATS_DIR}/features_train_h2.parquet"
test_path  = f"{FEATS_DIR}/features_test_h2.parquet"

df_train = pd.read_parquet(train_path)
df_test  = pd.read_parquet(test_path)

# Unique series id for global model
id_cols = ['origin','destination','hs6','trade_flow']
df_train['unique_id'] = df_train['origin'] + '|' + df_train['destination'] + '|' + df_train['hs6'] + '|' + df_train['trade_flow']
df_test['unique_id']  = df_test['origin']  + '|' + df_test['destination']  + '|' + df_test['hs6']  + '|' + df_test['trade_flow']

# NF expects ds/y
df_train = df_train.rename(columns={'month':'ds'})
df_test  = df_test.rename(columns={'month':'ds'})
df_train = df_train.sort_values(['unique_id','ds'])
df_test  = df_test.sort_values(['unique_id','ds'])

# Target frames
y_cols = ['unique_id','ds','y']
y_train = df_train[y_cols].copy()
y_test  = df_test[y_cols].copy()

# Optional exogs (off by default to keep things smooth)
DROP = set(['origin','destination','hs6','hs4','trade_flow','y','unique_id','ds'])
exog_cols = [c for c in df_train.columns if c not in DROP and pd.api.types.is_numeric_dtype(df_train[c])]
USE_EXOG = False  # <<< keep False for a first clean run
print("Exog candidates:", len(exog_cols))

Exog candidates: 34


In [14]:
# NHITS with smaller lookback and start-padding enabled
# (works with neuralforecast==1.6.4)

INPUT_SIZE = 12  # shorter history requirement (was 24)

common_kwargs = dict(
    h=H,
    input_size=INPUT_SIZE,
    max_steps=600,
    learning_rate=1e-3,
    scaler_type='robust',
    random_seed=42,
    start_padding_enabled=True,  # <-- allow left padding for short series
)

def make_nhits(futr_exog_list=None):
    # No trainer_kwargs to avoid version friction; Lightning will auto-detect GPU
    return NHITS(
        **common_kwargs,
        stack_types=['identity','identity'],
        n_blocks=[2, 2],
        mlp_units=[[256, 256], [256, 256]],
        futr_exog_list=futr_exog_list,
    )

print("NHITS builder ready. GPU auto-detect ->", DEVICE=="cuda", "| input_size:", INPUT_SIZE)

NHITS builder ready. GPU auto-detect -> True | input_size: 12


In [17]:
# Rolling CV WITHOUT exogs (robust to sparse folds)
from neuralforecast import NeuralForecast
import numpy as np

oof_list, cv_logs = [], []

for f_idx, (train_end, val_month) in enumerate(folds, 1):
    print(f"\n=== Fold {f_idx}: train<= {train_end}  |  validate= {val_month} (h=2) ===")

    # 1) Train subset
    y_tr = y_train[y_train['ds'] <= train_end].copy()  # ['unique_id','ds','y']
    if y_tr.empty:
        print(f"WARNING: Fold {f_idx} has no training rows. Skipping.")
        cv_logs.append({'fold': f_idx, 'train_end': train_end, 'val_month': val_month, 'smape': np.nan})
        continue

    # 1a) Keep ids with at least 1 point (padding handles short series)
    counts = y_tr.groupby('unique_id').size()
    keep_ids = counts[counts >= 1].index
    y_tr = y_tr[y_tr['unique_id'].isin(keep_ids)].copy()
    if y_tr.empty:
        print(f"WARNING: Fold {f_idx} has no series after min-length filter. Skipping.")
        cv_logs.append({'fold': f_idx, 'train_end': train_end, 'val_month': val_month, 'smape': np.nan})
        continue

    # 2) Future timestamps for next H months per kept series
    candidate = pd.concat([y_train[['unique_id','ds']], y_test[['unique_id','ds']]], ignore_index=True)
    X_future = (candidate[(candidate['unique_id'].isin(keep_ids)) & (candidate['ds'] > train_end)]
                .drop_duplicates(['unique_id','ds'])
                .sort_values(['unique_id','ds']))

    # 2a) Ensure exactly H future rows per series, and align y_tr to those ids
    X_future = (X_future.groupby('unique_id').head(H).reset_index(drop=True))
    fut_counts = X_future.groupby('unique_id').size()
    have_h = fut_counts[fut_counts >= H].index

    y_tr = y_tr[y_tr['unique_id'].isin(have_h)].copy()
    X_future = X_future[X_future['unique_id'].isin(have_h)].copy()

    if y_tr.empty or X_future.empty:
        print(f"WARNING: Fold {f_idx} has no train/future intersection with H={H}. Skipping.")
        cv_logs.append({'fold': f_idx, 'train_end': train_end, 'val_month': val_month, 'smape': np.nan})
        continue

    # 2b) Add dummy y required by nf.predict(df=...)
    X_future['y'] = np.nan

    # 3) Model + NF
    fold_model = make_nhits(futr_exog_list=None)
    nf = NeuralForecast(models=[fold_model], freq='MS')

    # 4) Fit
    nf.fit(df=y_tr, verbose=True)

    # 5) Predict H steps
    fcst = nf.predict(df=X_future)

    # 6) Normalize pred col
    pred_col = 'NHITS' if 'NHITS' in fcst.columns else ('NHiTS' if 'NHiTS' in fcst.columns else None)
    if pred_col is None:
        raise RuntimeError(f"Unexpected forecast columns: {fcst.columns.tolist()}")
    fcst = fcst.rename(columns={pred_col: 'y_pred_nhits'})

    # 7) Step-2 (t+2)
    step2 = (fcst.sort_values(['unique_id','ds'])
                  .groupby('unique_id').nth(1)
                  .reset_index())

    # 8) Join with truth (only for have_h ids)
    y_val = y_train[(y_train['unique_id'].isin(have_h)) & (y_train['ds'] == val_month)][['unique_id','ds','y']].copy()
    oof = y_val.merge(step2[['unique_id','ds','y_pred_nhits']], on=['unique_id','ds'], how='left')
    oof['fold'] = f_idx

    # 9) sMAPE
    fold_smape = smape(oof['y'].fillna(0), oof['y_pred_nhits'].fillna(0))
    print(f"Fold {f_idx} sMAPE: {fold_smape:.4f}")
    cv_logs.append({'fold': f_idx, 'train_end': train_end, 'val_month': val_month, 'smape': fold_smape})
    oof_list.append(oof)

# Save OOF + log
if oof_list:
    oof_all = pd.concat(oof_list, ignore_index=True).rename(columns={'ds':'month'})
    parts = oof_all['unique_id'].str.split('|', expand=True)
    oof_all['origin'] = parts[0]; oof_all['destination'] = parts[1]
    oof_all['hs6'] = parts[2];    oof_all['trade_flow'] = parts[3]
    oof_all = oof_all[['origin','destination','hs6','trade_flow','month','y','y_pred_nhits','fold']]

    oof_path = f"{OOF_DIR}/{MODEL_NAME}_oof.parquet"
    oof_all.to_parquet(oof_path, index=False); print("Saved OOF to:", oof_path)
else:
    print("No OOF rows to save—every fold was skipped.")

cv_df = pd.DataFrame(cv_logs); cv_df['run_id'] = RUN_ID
cv_df_path = f"{LOG_DIR}/cv_scores_{MODEL_NAME}_{RUN_ID}.csv"
cv_df.to_csv(cv_df_path, index=False); print("Saved CV log:", cv_df_path)
print("CV sMAPE mean (ignoring NaNs):", cv_df['smape'].mean(skipna=True))


=== Fold 1: train<= 2023-07-01  |  validate= 2023-09-01 (h=2) ===


INFO:lightning_fabric.utilities.seed:Seed set to 42


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Fold 2 sMAPE: 1.7960

=== Fold 3: train<= 2024-07-01  |  validate= 2024-09-01 (h=2) ===
Saved OOF to: /content/drive/MyDrive/ai4trade/predictions/oof/nhits_h2_oof.parquet
Saved CV log: /content/drive/MyDrive/ai4trade/logs/cv_scores_nhits_h2_20251024_1754.csv
CV sMAPE mean (ignoring NaNs): 1.6979646365927779


In [20]:
# === FINAL FIT (≤ FINAL_TRAIN_END) AND OCT-2025 FORECAST (t+2) — SAFE VERSION ===
from neuralforecast import NeuralForecast
import numpy as np
import pandas as pd

# 0) Helper: build month starts safely
def month_start(ts):
    # Return first day of the month for any date-like input
    return pd.to_datetime(ts).to_period('M').to_timestamp()

def build_future_grid(ids: pd.Series, anchor_date: str, H: int) -> pd.DataFrame:
    """
    Build exactly H future month-start timestamps strictly AFTER anchor_date,
    for each unique_id in ids.
    """
    anchor_ms = month_start(anchor_date)
    fut_months = [anchor_ms + pd.offsets.MonthBegin(k) for k in range(1, H+1)]
    grid = pd.DataFrame({
        "unique_id": np.repeat(ids.unique(), H),
        "ds": np.tile(fut_months, len(ids.unique()))
    })
    # NeuralForecast.predict(df=...) requires y column present (can be NaN)
    grid["y"] = np.nan
    return grid

# 1) Train through FINAL_TRAIN_END
y_full = y_train[y_train['ds'] <= FINAL_TRAIN_END].copy()
if y_full.empty:
    raise RuntimeError(f"No training data ≤ {FINAL_TRAIN_END}. Check your y_train dates.")

# Ensure ds is strictly month-start to match freq='MS'
y_full["ds"] = y_full["ds"].dt.to_period("M").dt.to_timestamp()

# 2) Build future grid for H=2 from calendar (no dependency on y_test)
X_future_full = build_future_grid(y_full["unique_id"], FINAL_TRAIN_END, H)

# Defensive checks
print("Diagnostics — final fit/forecast")
print("  Train rows:", len(y_full), " | Train series:", y_full['unique_id'].nunique())
print("  Future rows:", len(X_future_full), " | Future series:", X_future_full['unique_id'].nunique())
print("  Sample future months:", sorted(X_future_full['ds'].unique())[:5])

# 3) Model, fit, predict
final_model = make_nhits(futr_exog_list=None)
nf_final = NeuralForecast(models=[final_model], freq='MS')

# Fit requires columns ['unique_id','ds','y'] only
nf_final.fit(df=y_full[['unique_id','ds','y']], verbose=True)

# Predict requires ['unique_id','ds','y'] (y can be NaN)
fcst_full = nf_final.predict(df=X_future_full[['unique_id','ds','y']])

# 4) Normalize prediction column
pred_col = 'NHITS' if 'NHITS' in fcst_full.columns else ('NHiTS' if 'NHiTS' in fcst_full.columns else None)
if pred_col is None:
    raise RuntimeError(f"Unexpected forecast columns: {fcst_full.columns.tolist()}")
fcst_full = fcst_full.rename(columns={pred_col: 'y_pred_nhits'})

# 5) Take step-2 (t+2) and filter to FINAL_FORECAST_MONTH
fcst_full = fcst_full.sort_values(['unique_id','ds'])
step2_full = fcst_full.groupby('unique_id').nth(1).reset_index()

target_month = month_start(FINAL_FORECAST_MONTH)
step2_full = step2_full[step2_full['ds'] == target_month].copy()
if step2_full.empty:
    # Help debug if something mismatched
    print("WARN: step2_full is empty. Showing unique ds in forecast:", sorted(fcst_full['ds'].unique())[:5])
    raise RuntimeError(f"No t+2 predictions match FINAL_FORECAST_MONTH={FINAL_FORECAST_MONTH}. "
                       "Check calendar construction and ds alignment.")

# 6) Reattach identifiers and save
parts = step2_full['unique_id'].str.split('|', expand=True)
step2_full['origin'] = parts[0]
step2_full['destination'] = parts[1]
step2_full['hs6'] = parts[2]
step2_full['trade_flow'] = parts[3]
step2_full = step2_full.rename(columns={'ds':'month'})

final_fc_path = f"{FC_DIR}/{MODEL_NAME}_forecast.parquet"
step2_full[['origin','destination','hs6','trade_flow','month','y_pred_nhits']].to_parquet(final_fc_path, index=False)
print("Saved forecast to:", final_fc_path)

Diagnostics — final fit/forecast
  Train rows: 5979239  | Train series: 386394


INFO:lightning_fabric.utilities.seed:Seed set to 42


  Future rows: 772788  | Future series: 386394
  Sample future months: [Timestamp('2025-09-01 00:00:00'), Timestamp('2025-10-01 00:00:00')]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

WARN: step2_full is empty. Showing unique ds in forecast: [Timestamp('2025-11-01 00:00:00'), Timestamp('2025-12-01 00:00:00')]


RuntimeError: No t+2 predictions match FINAL_FORECAST_MONTH=2025-10-01. Check calendar construction and ds alignment.